In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import curio
import curio_http

ROOT = Path('../')
DATA_DIR = ROOT/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'test_watchlist.yaml')
w


{'email_address': 'brainbummer@mailinator.com',
 'name': 'test',
 'products':                          description stock_code
 0                     organic cheese     281739
 1                       GB chocolate     260803
 2                     Lupi olive oil     701829
 3      Earthcare double toilet paper     381895
 4                      Dijon mustard     700630
 5                whole grain mustard     700631
 6            Pics peanut butter 380g     360257
 7           Pics peanut butter 1000g     887052
 8                      coconut cream     271818
 9     Ecostore dishwash liquid 500mL     701108
 10  Earthwise dishwash liquid 1000mL     504018
 11                             phony      phony}

In [4]:
# Get and parse product info

code = w['products']['stock_code'].iat[-1]
response = cd.get_product(code)
cd.parse_product(response.text)

OrderedDict([('stock_code', None),
             ('name', None),
             ('description', None),
             ('size', None),
             ('sale_price', None),
             ('price', None),
             ('discount_percentage', None),
             ('unit_price', None),
             ('datetime', '2017-06-04T12:05:40')])

In [ ]:
# Collect info for several products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes)
f

In [ ]:
# Collect info for several products asynchronously

codes = w['products']['stock_code'].values
%time f = cd.collect_products_a(codes)
f

In [ ]:
# Filter sales

g = cd.filter_sales(f)
g

In [ ]:
# Email sales

domain = cd.get_secret('mailgun_domain')
key = cd.get_secret('mailgun_api_key')
cd.email(g, 'brainbummer@mailinator.com', domain, key)

In [ ]:
# Run pipeline: read watchlist, get product info, write product info, 
# filter sales, and email results

%time cd.run_pipeline(DATA_DIR/'test_watchlist.yaml', OUT_DIR, domain, key)